In [1]:
from get_embedding_function import get_embedding_function
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
    You are a medical assistant.
    Answer the question using ONLY the context below.
    If the answer is not contained in the context, say:
    "I dont know based on the provided documents."

    Context:    
    {context}

    ---

    Answer the question based on the above context: {question}
    """


/home/leonardo/miniconda3/envs/rag_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def query_rag(query_text: str):
    embedding_function = get_embedding_function()
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_function
    )
    
    results = db.similarity_search_with_score(query_text, k=5)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)
    print(response_text)
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [3]:
query_text = "What does the guideline say about treating malaria in pregnant women?"

query_rag(query_text)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1017.79it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/tmp/ipykernel_21863/3998500938.py:13: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  model = Ollama(model="mistral")


 The guideline states that during pregnancy, treatment for uncomplicated falciparum malaria should be an artemisinin-based combination therapy (ACT) given orally for 3 days. Primaquine, which is used to eliminate relapses of P. vivax and P. ovale, should not be given in pregnancy. For more specific information about pregnant women in high-risk areas, refer to the guide Essential obstetric and newborn care, MSF.
Response:  The guideline states that during pregnancy, treatment for uncomplicated falciparum malaria should be an artemisinin-based combination therapy (ACT) given orally for 3 days. Primaquine, which is used to eliminate relapses of P. vivax and P. ovale, should not be given in pregnancy. For more specific information about pregnant women in high-risk areas, refer to the guide Essential obstetric and newborn care, MSF.
Sources: ['../../data/guideline-170-en.pdf:185:1', '../../data/guideline-170-en.pdf:181:2', '../../data/guideline-170-en.pdf:181:1', '../../data/guideline-170-e

' The guideline states that during pregnancy, treatment for uncomplicated falciparum malaria should be an artemisinin-based combination therapy (ACT) given orally for 3 days. Primaquine, which is used to eliminate relapses of P. vivax and P. ovale, should not be given in pregnancy. For more specific information about pregnant women in high-risk areas, refer to the guide Essential obstetric and newborn care, MSF.'